In [74]:
import os
import sys
import spacy
import pandas as pd

sys.path.append('/home/rob/persyn/interaction')

from memory import LongTermMemory

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
BOT_NAME = os.environ["BOT_NAME"]
BOT_ID = os.environ["BOT_ID"]

ltm = LongTermMemory(
    bot_name=BOT_NAME,
    bot_id=BOT_ID,
    url=os.environ['ELASTIC_URL'],
    auth_name=os.environ["BOT_NAME"],
    auth_key=os.environ.get('ELASTIC_KEY', None),
    index_prefix=os.environ.get('ELASTIC_INDEX_PREFIX', BOT_NAME.lower()),
    verify_certs=True,
    version="v0"    
)

GET https://aramaki.persyn.io:9200/ [status:200 request:0.232s]
POST https://aramaki.persyn.io:9200/anna-conversations-v0/_search [status:200 request:0.059s]
POST https://aramaki.persyn.io:9200/anna-summaries-v0/_search [status:200 request:0.062s]
POST https://aramaki.persyn.io:9200/anna-entities-v0/_search [status:200 request:0.058s]
POST https://aramaki.persyn.io:9200/anna-relationships-v0/_search [status:200 request:0.064s]


In [11]:
convo = ltm.load_convo('https://unit-16.slack.com/','D02NK563878')

POST https://aramaki.persyn.io:9200/anna-conversations-v0/_search [status:200 request:0.076s]
POST https://aramaki.persyn.io:9200/anna-summaries-v0/_search [status:200 request:0.068s]


In [55]:
summaries = ltm.load_summaries('https://unit-16.slack.com/','D02NK563878', 1)

POST https://aramaki.persyn.io:9200/anna-summaries-v0/_search [status:200 request:0.064s]


In [56]:
summaries

["Anna's sister, Fiona, is doing well according to Anna. However, Anna feels like Fiona is always busy and never has time for her. Additionally, Anna feels like Fiona only talks about herself or her problems."]

In [60]:
lines = []

for line in summaries:
    for sent in nlp(line).sents:
        lines.append(["Narrator", sent])

for line in convo:
    speaker, rest = line.split(':', 1)
    for sent in nlp(rest.strip()).sents:
        lines.append([speaker, sent])

In [61]:
lines

[['Narrator', Anna's sister, Fiona, is doing well according to Anna.],
 ['Narrator',
  However, Anna feels like Fiona is always busy and never has time for her.],
 ['Narrator',
  Additionally, Anna feels like Fiona only talks about herself or her problems.],
 ['Rob', What is her problem anyway?],
 ['Anna', I don't know.],
 ['Anna', She's just been really distant lately.],
 ['Anna', Yeah, I'm not sure what's going on with her either.],
 ['Anna', I'm just really worried about her.],
 ['Anna', I don't know, she just seems really sad and scared.],
 ['Rob', Well I'm glad you're there for her.],
 ['Rob', Life isn't always easy.],
 ['Rob', Friends and family get us through.],
 ['Anna', Yeah, I feel really grateful to have her in my life.]]

In [76]:
entities = {}

# Find named entities, phrases and concepts
for speaker, doc in lines:
    dobj = None
    print(speaker, ':', doc)
    
    if speaker not in entities:
        entities[speaker] = {}
    
    spacy.displacy.render(doc, style="ent")
    
    for entity in set(doc.ents):
        if entity.text not in entities:
            entities[entity.text] = {}
            
    for token in doc:
        if token.dep_ == 'dobj':
            dobj = token
            break
    if dobj:
        print("Intent:", token.head.text + token.text.capitalize())

    print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
    print("")
    
    diag = {'pos': [], 'exp': [], 'token': []}
    for token in doc:
        diag['pos'].append(token.dep_)
        diag['exp'].append(spacy.explain(token.dep_))
        diag['token'].append(token.text)
#         print(token.dep_, spacy.explain(token.dep_), token.text)
    display(pd.DataFrame(diag))
    
    spacy.displacy.render(
        doc, 
        style="dep", 
        options={
            "compact":False,
            "fine_grained":True,
            "add_lemma":True,
            "collapse_punct":True,
            "collapse_phrases":True
        }
    )

Narrator : Anna's sister, Fiona, is doing well according to Anna.


Noun phrases: ["Anna's sister", 'Fiona', 'Anna']
Verbs: ['do', 'accord']



,pos,exp,token
0,poss,possession modifier,Anna
1,case,case marking,'s
2,nsubj,nominal subject,sister
3,punct,punctuation,","
4,appos,appositional modifier,Fiona
5,punct,punctuation,","
6,aux,auxiliary,is
7,ROOT,None,doing
8,advmod,adverbial modifier,well
9,prep,prepositional modifier,according


Narrator : However, Anna feels like Fiona is always busy and never has time for her.


Intent: hasTime
Noun phrases: ['Anna', 'Fiona', 'time', 'her']
Verbs: ['feel', 'have']



...

,pos,exp,token
0,nsubj,nominal subject,Friends
1,cc,coordinating conjunction,and
2,conj,conjunct,family
3,ROOT,None,get
4,dobj,direct object,us
5,prt,particle,through
6,punct,punctuation,.


Anna : Yeah, I feel really grateful to have her in my life.


/home/rob/persyn/interaction/env/lib/python3.8/site-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Intent: haveHer
Noun phrases: ['I', 'her', 'my life']
Verbs: ['feel', 'have']



,pos,exp,token
0,intj,interjection,Yeah
1,punct,punctuation,","
2,nsubj,nominal subject,I
3,ROOT,None,feel
4,advmod,adverbial modifier,really
5,acomp,adjectival complement,grateful
6,aux,auxiliary,to
7,xcomp,open clausal complement,have
8,dobj,direct object,her
9,prep,prepositional modifier,in


In [77]:
entities

{'Narrator': {}, 'Anna': {}, 'Fiona': {}, 'Rob': {}}

In [44]:
# j = doc.to_json()
# print(doc)
# print(' '.join([w['lemma'] for w in j['tokens']]))

print(doc)
print(' '.join([w.lemma_ for w in doc]))

Yeah, I feel really grateful to have her in my life.
yeah , I feel really grateful to have she in my life .


In [45]:
[n for n in doc.noun_chunks]

[I, her, my life]

In [46]:
for token in doc:
    print(token.dep_, token.text)

intj Yeah
punct ,
nsubj I
ROOT feel
advmod really
acomp grateful
aux to
xcomp have
dobj her
prep in
poss my
pobj life
punct .
